### Import required Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',50)

import warnings
warnings.filterwarnings(action='ignore')

### Import data

In [2]:
train = pd.read_csv('train.csv',index_col=0)
train.shape

(188533, 12)

In [3]:
test = pd.read_csv('test.csv',index_col=0)
test.shape

(125690, 11)

In [4]:
df = pd.concat([train,test],axis=0)

In [5]:
df.duplicated().sum()

0

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 314223 entries, 0 to 314222
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   brand         314223 non-null  object 
 1   model         314223 non-null  object 
 2   model_year    314223 non-null  int64  
 3   milage        314223 non-null  int64  
 4   fuel_type     305757 non-null  object 
 5   engine        314223 non-null  object 
 6   transmission  314223 non-null  object 
 7   ext_col       314223 non-null  object 
 8   int_col       314223 non-null  object 
 9   accident      310139 non-null  object 
 10  clean_title   278565 non-null  object 
 11  price         188533 non-null  float64
dtypes: float64(1), int64(2), object(9)
memory usage: 31.2+ MB


In [7]:
df.isna().sum()

brand                0
model                0
model_year           0
milage               0
fuel_type         8466
engine               0
transmission         0
ext_col              0
int_col              0
accident          4084
clean_title      35658
price           125690
dtype: int64

## Feature Engineering

### Brand

In [8]:
df.brand.value_counts()

brand
Ford             38353
Mercedes-Benz    32047
BMW              28314
Chevrolet        27033
Audi             18176
Porsche          17765
Land             15873
Toyota           14934
Lexus            14372
Jeep             10886
Cadillac          7733
RAM               7002
Nissan            6604
Tesla             6125
INFINITI          5399
GMC               5309
Dodge             5204
Mazda             4489
Kia               4247
Lincoln           4016
Subaru            3972
Acura             3849
Honda             3537
Hyundai           3353
Volkswagen        2987
Jaguar            2253
Bentley           1944
MINI              1761
Genesis           1612
Buick             1591
Maserati          1558
Lamborghini       1356
Volvo             1299
Chrysler          1241
Alfa              1149
Rivian             980
Rolls-Royce        944
Mitsubishi         918
Pontiac            879
Hummer             862
Ferrari            610
McLaren            408
Aston              382
Satur

In [9]:
df = pd.concat([df,pd.get_dummies(df.brand,dtype='int')],axis=1)
df.drop('brand',axis=1,inplace=True)

### Model

In [10]:
df.model.value_counts()

model
F-150 XLT                   4868
M3 Base                     3641
Camaro 2SS                  2820
M4 Base                     2627
Mustang GT Premium          2464
                            ... 
X5 xDrive40e                   2
Forte LX                       2
V60 T6 R-Design Platinum       2
X5 3.0i                        1
allroad 2.7T                   1
Name: count, Length: 1898, dtype: int64

In [11]:
model_list = list(df.model)

d = {}
for i in model_list:
    if i not in d:
        d[i] = 1
    else:
        d[i]+=1

In [12]:
model_value_counts = pd.DataFrame(set(df.model.value_counts().values))[0]

q10 = model_value_counts.quantile(0.10)
q20 = model_value_counts.quantile(0.20)
q30 = model_value_counts.quantile(0.30)
q40 = model_value_counts.quantile(0.40)
q50 = model_value_counts.quantile(0.50)
q60 = model_value_counts.quantile(0.60)
q70 = model_value_counts.quantile(0.70)
q80 = model_value_counts.quantile(0.80)
q90 = model_value_counts.quantile(0.90)

models = []
for i in model_list:
    if d[i] < q10:
        models.append('model_1')
    elif d[i] >= q10 and d[i] < q20:
        models.append('model_2')
    elif d[i] >= q20 and d[i] < q30:
        models.append('model_3')
    elif d[i] >= q30 and d[i] < q40:
        models.append('model_4')
    elif d[i] >= q40 and d[i] < q50:
        models.append('model_5')
    elif d[i] >= q50 and d[i] < q60:
        models.append('model_6')
    elif d[i] >= q60 and d[i] < q70:
        models.append('model_7')
    elif d[i] >= q70 and d[i] < q80:
        models.append('model_8')
    elif d[i] >= q80 and d[i] < q90:
        models.append('model_9')
    elif d[i] >= q90:
        models.append('model_10')

In [13]:
df = pd.concat([df,pd.DataFrame(models,columns=['models'])],axis=1)
df.drop('model',axis=1,inplace=True)

In [14]:
df = pd.concat([df,pd.get_dummies(df.models,dtype='int')],axis=1)
df.drop('models',axis=1,inplace=True)

### Model Year

In [15]:
df.model_year.value_counts()

model_year
2021    30234
2018    27559
2020    26565
2022    26163
2019    25393
2016    22758
2017    21208
2015    18925
2023    14552
2014    13964
2013    11465
2011    10497
2012    10310
2008     8534
2007     8224
2010     7038
2005     5234
2006     4722
2009     4595
2004     4183
2003     3175
2001     2583
2002     2162
1999      952
2000      911
1998      590
1997      526
1996      309
2024      277
1993      228
1994      175
1995      171
1992       21
1974       18
1982        1
1988        1
Name: count, dtype: int64

In [16]:
def bucket2(a):
    if a<=1992:
        return 'Old1'
    elif a>1992 and a<=1999:
        return 'Old2'
    elif a>1999 and a<=2008:
        return 'Old3'
    elif a>2008 and a<=2015:
        return 'Old4'
    elif a>2015:
        return 'Old5'

In [17]:
df.model_year = df.model_year.apply(bucket2)

In [18]:
df = pd.concat([df,pd.get_dummies(df.model_year,dtype='int')],axis=1)
df.drop('model_year',axis=1,inplace=True)

### Milage

In [19]:
df.milage.value_counts()

milage
60000     2182
170000    2165
120000    1920
110000    1918
130000    1634
          ... 
70169        1
151775       1
87582        1
133815       1
21436        1
Name: count, Length: 8440, dtype: int64

In [20]:
df.milage.describe()

count    314223.000000
mean      65840.210567
std       49969.067924
min         100.000000
25%       24213.000000
50%       57690.000000
75%       95500.000000
max      405000.000000
Name: milage, dtype: float64

In [21]:
# Descile Partition
q10 = df.milage.quantile(0.10)
q20 = df.milage.quantile(0.20)
q30 = df.milage.quantile(0.30)
q40 = df.milage.quantile(0.40)
q50 = df.milage.quantile(0.50)
q60 = df.milage.quantile(0.60)
q70 = df.milage.quantile(0.70)
q80 = df.milage.quantile(0.80)
q90 = df.milage.quantile(0.90)

def bucket(a):
    if a < q10:
        return 'm10'
    elif a >= q10 and a < q20:
        return 'm20'
    elif a >= q20 and a < q30:
        return 'm30'
    elif a >= q30 and a < q40:
        return 'm40'
    elif a >= q40 and a < q50:
        return 'm50'
    elif a >= q50 and a < q60:
        return 'm60'
    elif a >= q60 and a < q70:
        return 'm70'
    elif a >= q70 and a < q80:
        return 'm80'
    elif a >= q80 and a < q90:
        return 'm90'
    elif a >= q90:
        return 'm100'

In [22]:
df.milage = df.milage.apply(bucket)

In [23]:
df = pd.concat([df,pd.get_dummies(df.milage,dtype='int')],axis=1)
df.drop('milage',axis=1,inplace=True)

### Fuel Type

In [24]:
df.fuel_type.value_counts()

fuel_type
Gasoline          276473
Hybrid             11508
E85 Flex Fuel       8929
Diesel              6641
–                   1319
Plug-In Hybrid       858
not supported         29
Name: count, dtype: int64

In [25]:
df.fuel_type.replace('–','fuel_type_Missing',inplace=True)

In [26]:
df.fuel_type = df.fuel_type.apply(lambda x: x if isinstance(x,str) else 'fuel_type_Missing')

In [27]:
df = pd.concat([df,pd.get_dummies(df.fuel_type,dtype='int')],axis=1)
df.drop('fuel_type',axis=1,inplace=True)

### Engine

In [28]:
df.engine.value_counts()

engine
355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel           5651
240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel           4868
420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel           4694
2.0L I4 16V GDI DOHC Turbo                             4513
375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel          4090
                                                       ... 
70.0HP 1.0L 3 Cylinder Engine Gasoline Fuel               3
111.0HP Electric Motor Electric Fuel System               3
139.0HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas       3
78.0HP 1.2L 3 Cylinder Engine Gasoline Fuel               2
313.0HP 2.0L 4 Cylinder Engine Plug-In Electric/Gas       1
Name: count, Length: 1118, dtype: int64

In [29]:
df.engine.replace('–','engine_Missing_value',inplace=True)

In [30]:
engine_list = list(df.engine)

d = {}
for i in engine_list:
    if i not in d:
        d[i] = 1
    else:
        d[i]+=1

In [31]:
engine_value_counts = pd.DataFrame(set(df.engine.value_counts().values))[0]

q10 = engine_value_counts.quantile(0.10)
q20 = engine_value_counts.quantile(0.20)
q30 = engine_value_counts.quantile(0.30)
q40 = engine_value_counts.quantile(0.40)
q50 = engine_value_counts.quantile(0.50)
q60 = engine_value_counts.quantile(0.60)
q70 = engine_value_counts.quantile(0.70)
q80 = engine_value_counts.quantile(0.80)
q90 = engine_value_counts.quantile(0.90)

engines = []
for i in engine_list:
    if d[i] < q10:
        engines.append('engine_1')
    elif d[i] >= q10 and d[i] < q20:
        engines.append('engine_2')
    elif d[i] >= q20 and d[i] < q30:
        engines.append('engine_3')
    elif d[i] >= q30 and d[i] < q40:
        engines.append('engine_4')
    elif d[i] >= q40 and d[i] < q50:
        engines.append('engine_5')
    elif d[i] >= q50 and d[i] < q60:
        engines.append('engine_6')
    elif d[i] >= q60 and d[i] < q70:
        engines.append('engine_7')
    elif d[i] >= q70 and d[i] < q80:
        engines.append('engine_8')
    elif d[i] >= q80 and d[i] < q90:
        engines.append('engine_9')
    elif d[i] >= q90:
        engines.append('engine_10')

In [32]:
df = pd.concat([df,pd.DataFrame(engines,columns=['engines'])],axis=1)
df.drop('engine',axis=1,inplace=True)

In [33]:
df = pd.concat([df,pd.get_dummies(df.engines,dtype='int')],axis=1)
df.drop('engines',axis=1,inplace=True)

### Transmission

In [34]:
df.transmission.value_counts()

transmission
A/T                                                   83036
8-Speed A/T                                           34586
Transmission w/Dual Shift Mode                        32079
6-Speed A/T                                           30105
6-Speed M/T                                           19863
7-Speed A/T                                           18498
Automatic                                             17866
8-Speed Automatic                                     13977
10-Speed A/T                                          13347
9-Speed A/T                                            6450
5-Speed A/T                                            5458
10-Speed Automatic                                     5268
6-Speed Automatic                                      4690
4-Speed A/T                                            4212
5-Speed M/T                                            4048
9-Speed Automatic                                      3878
CVT Transmission           

In [35]:
df.transmission.replace('–','transmission_Missing_value',inplace=True)

In [36]:
transmission_list = list(df.transmission)

d = {}
for i in transmission_list:
    if i not in d:
        d[i] = 1
    else:
        d[i]+=1

In [37]:
transmission_value_counts = pd.DataFrame(set(df.transmission.value_counts().values))[0]

q10 = transmission_value_counts.quantile(0.10)
q20 = transmission_value_counts.quantile(0.20)
q30 = transmission_value_counts.quantile(0.30)
q40 = transmission_value_counts.quantile(0.40)
q50 = transmission_value_counts.quantile(0.50)
q60 = transmission_value_counts.quantile(0.60)
q70 = transmission_value_counts.quantile(0.70)
q80 = transmission_value_counts.quantile(0.80)
q90 = transmission_value_counts.quantile(0.90)

transmissions = []
for i in transmission_list:
    if d[i] < q10:
        transmissions.append('transmission_1')
    elif d[i] >= q10 and d[i] < q20:
        transmissions.append('transmission_2')
    elif d[i] >= q20 and d[i] < q30:
        transmissions.append('transmission_3')
    elif d[i] >= q30 and d[i] < q40:
        transmissions.append('transmission_4')
    elif d[i] >= q40 and d[i] < q50:
        transmissions.append('transmission_5')
    elif d[i] >= q50 and d[i] < q60:
        transmissions.append('transmission_6')
    elif d[i] >= q60 and d[i] < q70:
        transmissions.append('transmission_7')
    elif d[i] >= q70 and d[i] < q80:
        transmissions.append('transmission_8')
    elif d[i] >= q80 and d[i] < q90:
        transmissions.append('transmission_9')
    elif d[i] >= q90:
        transmissions.append('transmission_10')

In [38]:
df = pd.concat([df,pd.DataFrame(transmissions,columns=['transmissions'])],axis=1)
df.drop('transmission',axis=1,inplace=True)

In [39]:
df = pd.concat([df,pd.get_dummies(df.transmissions,dtype='int')],axis=1)
df.drop('transmissions',axis=1,inplace=True)

### Color

In [40]:
df.ext_col.value_counts()

ext_col
Black                                 81028
White                                 73040
Gray                                  42193
Silver                                28343
Blue                                  24272
                                      ...  
Granite Crystal Metallic Clearcoat        8
BLUE                                      7
Nautical Blue Pearl                       7
Mango Tango Pearlcoat                     5
GT SILVER                                 5
Name: count, Length: 319, dtype: int64

In [41]:
df.ext_col.replace('–','color_Missing_value',inplace=True)

In [42]:
ext_col_list = list(df.ext_col)

d = {}
for i in ext_col_list:
    if i not in d:
        d[i] = 1
    else:
        d[i]+=1

In [43]:
ext_col_value_counts = pd.DataFrame(set(df.ext_col.value_counts().values))[0]

q10 = ext_col_value_counts.quantile(0.10)
q20 = ext_col_value_counts.quantile(0.20)
q30 = ext_col_value_counts.quantile(0.30)
q40 = ext_col_value_counts.quantile(0.40)
q50 = ext_col_value_counts.quantile(0.50)
q60 = ext_col_value_counts.quantile(0.60)
q70 = ext_col_value_counts.quantile(0.70)
q80 = ext_col_value_counts.quantile(0.80)
q90 = ext_col_value_counts.quantile(0.90)

ext_cols = []
for i in ext_col_list:
    if d[i] < q10:
        ext_cols.append('ext_col_1')
    elif d[i] >= q10 and d[i] < q20:
        ext_cols.append('ext_col_2')
    elif d[i] >= q20 and d[i] < q30:
        ext_cols.append('ext_col_3')
    elif d[i] >= q30 and d[i] < q40:
        ext_cols.append('ext_col_4')
    elif d[i] >= q40 and d[i] < q50:
        ext_cols.append('ext_col_5')
    elif d[i] >= q50 and d[i] < q60:
        ext_cols.append('ext_col_6')
    elif d[i] >= q60 and d[i] < q70:
        ext_cols.append('ext_col_7')
    elif d[i] >= q70 and d[i] < q80:
        ext_cols.append('ext_col_8')
    elif d[i] >= q80 and d[i] < q90:
        ext_cols.append('ext_col_9')
    elif d[i] >= q90:
        ext_cols.append('ext_col_10')

In [44]:
df = pd.concat([df,pd.DataFrame(ext_cols,columns=['ext_cols'])],axis=1)
df.drop('ext_col',axis=1,inplace=True)

In [45]:
df = pd.concat([df,pd.get_dummies(df.ext_cols,dtype='int')],axis=1)
df.drop('ext_cols',axis=1,inplace=True)

### In Color

In [46]:
df.int_col.value_counts()

int_col
Black           179287
Beige            40990
Gray             35381
Brown             9721
Red               8641
                 ...  
WHITE               14
Carbon Black        12
Bianco Polar        11
Cobalt Blue          9
ORANGE               6
Name: count, Length: 156, dtype: int64

In [47]:
df.int_col.replace('–','int_col_Missing_value',inplace=True)

In [48]:
int_col_list = list(df.int_col)

d = {}
for i in int_col_list:
    if i not in d:
        d[i] = 1
    else:
        d[i]+=1

In [49]:
int_col_value_counts = pd.DataFrame(set(df.int_col.value_counts().values))[0]

q10 = int_col_value_counts.quantile(0.10)
q20 = int_col_value_counts.quantile(0.20)
q30 = int_col_value_counts.quantile(0.30)
q40 = int_col_value_counts.quantile(0.40)
q50 = int_col_value_counts.quantile(0.50)
q60 = int_col_value_counts.quantile(0.60)
q70 = int_col_value_counts.quantile(0.70)
q80 = int_col_value_counts.quantile(0.80)
q90 = int_col_value_counts.quantile(0.90)

int_cols = []
for i in int_col_list:
    if d[i] < q10:
        int_cols.append('int_col_1')
    elif d[i] >= q10 and d[i] < q20:
        int_cols.append('int_col_2')
    elif d[i] >= q20 and d[i] < q30:
        int_cols.append('int_col_3')
    elif d[i] >= q30 and d[i] < q40:
        int_cols.append('int_col_4')
    elif d[i] >= q40 and d[i] < q50:
        int_cols.append('int_col_5')
    elif d[i] >= q50 and d[i] < q60:
        int_cols.append('int_col_6')
    elif d[i] >= q60 and d[i] < q70:
        int_cols.append('int_col_7')
    elif d[i] >= q70 and d[i] < q80:
        int_cols.append('int_col_8')
    elif d[i] >= q80 and d[i] < q90:
        int_cols.append('int_col_9')
    elif d[i] >= q90:
        int_cols.append('int_col_10')

In [50]:
df = pd.concat([df,pd.DataFrame(int_cols,columns=['int_cols'])],axis=1)
df.drop('int_col',axis=1,inplace=True)

In [51]:
df = pd.concat([df,pd.get_dummies(df.int_cols,dtype='int')],axis=1)
df.drop('int_cols',axis=1,inplace=True)

### Accident

In [52]:
df.accident.value_counts()

accident
None reported                             240777
At least 1 accident or damage reported     69362
Name: count, dtype: int64

In [53]:
df.clean_title.fillna('accident_Missing_value',inplace=True)

In [54]:
df = pd.concat([df,pd.get_dummies(df.accident,dtype='int')],axis=1)
df.drop('accident',axis=1,inplace=True)

### Clean Title

In [55]:
df.clean_title.value_counts()

clean_title
Yes                       278565
accident_Missing_value     35658
Name: count, dtype: int64

In [56]:
df.clean_title.fillna('clean_title_Missing_value',inplace=True)

In [57]:
df = pd.concat([df,pd.get_dummies(df.clean_title,dtype='int')],axis=1)
df.drop('clean_title',axis=1,inplace=True)

### Split back the data

In [58]:
train = df[~df.price.isna()]
test = df[df.price.isna()]

In [59]:
test.drop('price',axis=1,inplace=True)

## Model

In [60]:
xtrain = train.drop('price',axis=1)
ytrain = train['price']
xtest = test

In [61]:
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(xtrain,ytrain)
ypred = xgb.predict(xtest)

In [62]:
pd.concat([pd.DataFrame(test.index),pd.DataFrame(ypred,columns=['price'])],axis=1).to_csv('Final.csv',index=False)